In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!pip install fuzzywuzzy

In [5]:
import pickle

import numpy as np
import gensim
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
from tqdm import tqdm
from scipy.stats import skew, kurtosis
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis
from nltk import word_tokenize
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [231]:
import keras, nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from sklearn.metrics import f1_score

##Data Cleaning

In [138]:
def review_to_wordlist(review, remove_stopwords=True):
  if(type(review) is str):
    words = review.lower().split()
  else:
    words = str(review).lower().split()
  if remove_stopwords:
      stops = set(stopwords.words("english"))
      words = [w for w in words if not w in stops]
  
    
  review_text = " ".join(words)
  review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
  review_text = re.sub(r"\'s", " 's ", review_text)
  review_text = re.sub(r"\'ve", " 've ", review_text)
  review_text = re.sub(r"n\'t", " 't ", review_text)
  review_text = re.sub(r"\'re", " 're ", review_text)
  review_text = re.sub(r"\'d", " 'd ", review_text)
  review_text = re.sub(r"\'ll", " 'll ", review_text)
  review_text = re.sub(r",", " ", review_text)
  review_text = re.sub(r"\.", " ", review_text)
  review_text = re.sub(r"!", " ", review_text)
  review_text = re.sub(r"\(", " ( ", review_text)
  review_text = re.sub(r"\)", " ) ", review_text)
  review_text = re.sub(r"\?", " ", review_text)
  review_text = re.sub(r"\s{2,}", " ", review_text)
  
  words = review_text.split()
  # Shorten words to their stems
  stemmer = SnowballStemmer('english')
  stemmed_words = [stemmer.stem(word) for word in words]
  
  review_text = " ".join(stemmed_words)
  
  # Return a list of words
  return(review_text)

def process_questions(question_list, questions, question_list_name):
# function to transform questions and display progress
  for question in questions:
    question_list.append(review_to_wordlist(question))
    if len(question_list) % 100000 == 0:
      progress = len(question_list)/len(data) * 100
      print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

  print("{} is {}% complete.".format(question_list_name, 100))

In [ ]:
def norm_wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return norm_model.wmdistance(s1, s2)


def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

def wmd(s1, s2):
    s1 = str(s1).lower().split()
    s2 = str(s2).lower().split()
    stop_words = stopwords.words('english')
    s1 = [w for w in s1 if w not in stop_words]
    s2 = [w for w in s2 if w not in stop_words]
    return model.wmdistance(s1, s2)


In [ ]:


data = pd.read_csv('/content/drive/MyDrive/questions.csv')
data = data.drop(['id', 'qid1', 'qid2'], axis=1)

In [ ]:

data['len_q1'] = data.question1.apply(lambda x: len(str(x)))
data['len_q2'] = data.question2.apply(lambda x: len(str(x)))
data['diff_len'] = data.len_q1 - data.len_q2
data['len_char_q1'] = data.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_char_q2'] = data.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data['len_word_q1'] = data.question1.apply(lambda x: len(str(x).split()))
data['len_word_q2'] = data.question2.apply(lambda x: len(str(x).split()))
data['common_words'] = data.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
data['fuzz_qratio'] = data.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_WRatio'] = data.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_ratio'] = data.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_token_set_ratio'] = data.apply(lambda x: fuzz.partial_token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_partial_token_sort_ratio'] = data.apply(lambda x: fuzz.partial_token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_set_ratio'] = data.apply(lambda x: fuzz.token_set_ratio(str(x['question1']), str(x['question2'])), axis=1)
data['fuzz_token_sort_ratio'] = data.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)

In [ ]:
data

,question1,question2,is_duplicate,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,66,57,9,20,20,14,12,10,93,95,98,100,88,100,93
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,51,88,-37,21,29,8,13,4,66,86,73,100,73,86,63
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,73,59,14,25,24,14,10,4,43,60,41,100,71,63,63
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,50,65,-15,19,26,11,9,0,9,27,20,32,30,28,24
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,76,39,37,25,18,13,7,2,35,86,54,100,67,67,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404346,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,85,79,6,24,24,14,13,11,91,92,84,100,83,97,88
404347,Do you believe there is life after death?,Is it true that there is life after death?,1,41,42,-1,17,13,8,9,5,72,75,76,100,70,79,69
404348,What is one coin?,What's this coin?,0,17,17,0,11,11,4,3,1,75,75,76,100,75,75,75
404349,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,94,127,-33,26,27,17,25,1,30,35,34,100,36,37,35


In [ ]:
import gensim
google_news_path = "/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"

# Load google news vecs in gensim
model = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)
# model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin.gz', binary=True)
data['wmd'] = data.apply(lambda x: wmd(x['question1'], x['question2']), axis=1)


norm_model = gensim.models.KeyedVectors.load_word2vec_format(google_news_path, binary=True)
norm_model.init_sims(replace=True)
data['norm_wmd'] = data.apply(lambda x: norm_wmd(x['question1'], x['question2']), axis=1)

question1_vectors = np.zeros((data.shape[0], 300))
error_count = 0



In [ ]:
for i, q in tqdm(enumerate(data.question1.values)):
    question1_vectors[i, :] = sent2vec(q)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
246635it [00:58, 4192.14it/s]

In [ ]:


question2_vectors  = np.zeros((data.shape[0], 300))
for i, q in tqdm(enumerate(data.question2.values)):
    question2_vectors[i, :] = sent2vec(q)

data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(question1_vectors),
                                                          np.nan_to_num(question2_vectors))]

data['skew_q1vec'] = [skew(x) for x in np.nan_to_num(question1_vectors)]
data['skew_q2vec'] = [skew(x) for x in np.nan_to_num(question2_vectors)]
data['kur_q1vec'] = [kurtosis(x) for x in np.nan_to_num(question1_vectors)]
data['kur_q2vec'] = [kurtosis(x) for x in np.nan_to_num(question2_vectors)]

pickle.dump(question1_vectors, open('data/q1_w2v.pkl', 'wb'), -1)
pickle.dump(question2_vectors, open('data/q2_w2v.pkl', 'wb'), -1)

data.to_csv('/content/drive/MyDrive/quora_features.csv', index=False)

In [ ]:
def common_bigrams_count(q1, q2):
    tokens_1 = word_tokenize(q1)
    tokens_2 = word_tokenize(q2)
    if len(tokens_1) <2  or len(tokens_2) <2 :
      return 0
    bigrams_1 = nltk.bigrams(tokens_1)
    bigrams_2 = nltk.bigrams(tokens_2)

    

    frequency_1 = nltk.FreqDist(bigrams_1)
    frequency_2 = nltk.FreqDist(bigrams_2)

    count = 0
    common_pair=[]
    for pair in frequency_1.keys():
      if pair in frequency_2.keys():
        count=count+1
        common_pair.append(pair)
    
    # print(common_pair)
  
  
    return count

In [ ]:
def common_trigrams_count(q1, q2):
    tokens_1 = custom_tokenize(q1)
    tokens_2 = custom_tokenize(q2)

    if len(tokens_1) < 3 or len(tokens_2) < 3:
      return 0

    trigrams_1 = nltk.trigrams(tokens_1)
    trigrams_2 = nltk.trigrams(tokens_2)

    # print(trigrams_1)
    # print(trigrams_2)

    frequency_1 = nltk.FreqDist(trigrams_1)
    frequency_2 = nltk.FreqDist(trigrams_2)

    # print(frequency_1)
    # print(frequency_2)

    count = 0
    common_pair=[]
    for pair in frequency_1.keys():
      if pair in frequency_2.keys():
        count=count+1
        common_pair.append(pair)
    
    # print(common_pair)
  
  
    return count

In [ ]:
data['common_bigrams'] = data.apply(lambda x: common_bigrams_count(str(x['question1']), str(x['question2'])), axis=1)

In [ ]:
data['common_trigrams'] = data.apply(lambda x: common_bigrams_count(str(x['question1']), str(x['question2'])), axis=1)

In [ ]:
data.to_csv('/content/drive/MyDrive/quora_features.csv', index=False)

In [6]:
data = pd.read_csv('/content/drive/MyDrive/quora_features.csv')

In [7]:
data

,question1,question2,is_duplicate,wmd,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,common_bigrams,common_trigrams
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.564615,0.217555,0.068972,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301,66,57,9,20,20,14,12,10,93,95,98,100,88,100,93,11,11
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,3.772346,1.368796,0.512164,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444,51,88,-37,21,29,8,13,4,66,86,73,100,73,86,63,3,3
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1.780585,0.639209,0.222009,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131,73,59,14,25,24,14,10,4,43,60,41,100,71,63,63,1,1
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,3.741994,1.263719,0.650411,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568,50,65,-15,19,26,11,9,0,9,27,20,32,30,28,24,0,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3.659165,1.240908,0.369993,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876,76,39,37,25,18,13,7,2,35,86,54,100,67,67,47,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404346,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,0.904016,0.227233,0.131646,7.122625,1.0,118.479153,0.513120,0.231429,0.262434,0.014864,0.134802,-0.084945,-0.477739,85,79,6,24,24,14,13,11,91,92,84,100,83,97,88,9,9
404347,Do you believe there is life after death?,Is it true that there is life after death?,1,1.306167,0.590033,0.129039,7.146889,1.0,119.485637,0.508014,0.226945,0.263738,-0.002918,0.038642,-0.147208,-0.260159,41,42,-1,17,13,8,9,5,72,75,76,100,70,79,69,5,5
404348,What is one coin?,What's this coin?,0,3.656829,1.382975,0.069016,5.065351,1.0,91.936365,0.371527,0.170819,0.182876,-0.193922,-0.147340,-0.279527,-0.397618,17,17,0,11,11,4,3,1,75,75,76,100,75,75,75,1,1
404349,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,3.575980,1.313263,0.708233,16.438430,1.0,194.867266,1.190154,0.540202,0.737343,-0.018571,-0.155503,-0.382487,-0.162977,94,127,-33,26,27,17,25,1,30,35,34,100,36,37,35,0,0


In [23]:
fs_1 = ['len_q1', 'len_q2', 'diff_len', 'len_char_q1', 
        'len_char_q2', 'len_word_q1', 'len_word_q2',     
        'common_words']

In [24]:
data[fs_1]

,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words
0,66,57,9,20,20,14,12,10
1,51,88,-37,21,29,8,13,4
2,73,59,14,25,24,14,10,4
3,50,65,-15,19,26,11,9,0
4,76,39,37,25,18,13,7,2
...,...,...,...,...,...,...,...,...
404346,85,79,6,24,24,14,13,11
404347,41,42,-1,17,13,8,9,5
404348,17,17,0,11,11,4,3,1
404349,94,127,-33,26,27,17,25,1


In [25]:
fs_2 = ['fuzz_qratio', 'fuzz_WRatio', 'fuzz_partial_ratio', 
       'fuzz_partial_token_set_ratio', 'fuzz_partial_token_sort_ratio',
       'fuzz_token_set_ratio', 'fuzz_token_sort_ratio']

In [26]:
data[fs_2]

,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio
0,93,95,98,100,88,100,93
1,66,86,73,100,73,86,63
2,43,60,41,100,71,63,63
3,9,27,20,32,30,28,24
4,35,86,54,100,67,67,47
...,...,...,...,...,...,...,...
404346,91,92,84,100,83,97,88
404347,72,75,76,100,70,79,69
404348,75,75,76,100,75,75,75
404349,30,35,34,100,36,37,35


In [87]:
fs4 = ['cosine_distance', 'cityblock_distance', 
         'jaccard_distance', 'canberra_distance', 
         'euclidean_distance', 'minkowski_distance',
         'braycurtis_distance']
data[fs4]

,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance
0,0.068972,5.081614,1.0,94.023324,0.371408,0.168999,0.186557
1,0.512164,14.195119,1.0,177.588090,1.012091,0.455910,0.592655
2,0.222009,9.055989,1.0,135.988707,0.666346,0.307828,0.342306
3,0.650411,15.987437,1.0,192.237828,1.140536,0.506028,0.692421
4,0.369993,12.103178,1.0,161.408435,0.860225,0.382770,0.480633
...,...,...,...,...,...,...,...
404346,0.131646,7.122625,1.0,118.479153,0.513120,0.231429,0.262434
404347,0.129039,7.146889,1.0,119.485637,0.508014,0.226945,0.263738
404348,0.069016,5.065351,1.0,91.936365,0.371527,0.170819,0.182876
404349,0.708233,16.438430,1.0,194.867266,1.190154,0.540202,0.737343


In [139]:
questions1 = []     
process_questions(questions1, data.question1, "questions1")

questions1 is 24.7% complete.
questions1 is 49.5% complete.
questions1 is 74.2% complete.
questions1 is 98.9% complete.
questions1 is 100% complete.


In [140]:
questions2 = []     
process_questions(questions2, data.question2, "questions2")

questions2 is 24.7% complete.
questions2 is 49.5% complete.
questions2 is 74.2% complete.
questions2 is 98.9% complete.
questions2 is 100% complete.


In [142]:
data["processed_q1"]=np.array(questions1)
data["processed_q2"]=np.array(questions2)

In [143]:
data

,question1,question2,is_duplicate,wmd,norm_wmd,cosine_distance,cityblock_distance,jaccard_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance,skew_q1vec,skew_q2vec,kur_q1vec,kur_q2vec,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_partial_token_set_ratio,fuzz_partial_token_sort_ratio,fuzz_token_set_ratio,fuzz_token_sort_ratio,common_bigrams,common_trigrams,processed_q1,processed_q2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.564615,0.217555,0.068972,5.081614,1.0,94.023324,0.371408,0.168999,0.186557,0.031817,-0.091902,0.050416,0.337301,66,57,9,20,20,14,12,10,93,95,98,100,88,100,93,11,11,step step guid invest share market india,step step guid invest share market
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,3.772346,1.368796,0.512164,14.195119,1.0,177.588090,1.012091,0.455910,0.592655,0.008735,0.094704,0.284010,-0.034444,51,88,-37,21,29,8,13,4,66,86,73,100,73,86,63,3,3,stori kohinoor ( koh i noor ) diamond,would happen indian govern stole kohinoor ( ko...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1.780585,0.639209,0.222009,9.055989,1.0,135.988707,0.666346,0.307828,0.342306,0.239752,0.144554,0.026759,-0.474131,73,59,14,25,24,14,10,4,43,60,41,100,71,63,63,1,1,increas speed internet connect use vpn,internet speed increas hack dns
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,3.741994,1.263719,0.650411,15.987437,1.0,192.237828,1.140536,0.506028,0.692421,-0.002527,0.069649,-0.244560,-0.265568,50,65,-15,19,26,11,9,0,9,27,20,32,30,28,24,0,0,mental lone solv it,find remaind math 23 24 math divid 24 23
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,3.659165,1.240908,0.369993,12.103178,1.0,161.408435,0.860225,0.382770,0.480633,-0.133849,0.114777,0.217900,-0.338876,76,39,37,25,18,13,7,2,35,86,54,100,67,67,47,0,0,one dissolv water quik sugar salt methan carbo...,fish would surviv salt water
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404346,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0,0.904016,0.227233,0.131646,7.122625,1.0,118.479153,0.513120,0.231429,0.262434,0.014864,0.134802,-0.084945,-0.477739,85,79,6,24,24,14,13,11,91,92,84,100,83,97,88,9,9,mani keyword racket program languag latest ver...,mani keyword perl program languag latest version
404347,Do you believe there is life after death?,Is it true that there is life after death?,1,1.306167,0.590033,0.129039,7.146889,1.0,119.485637,0.508014,0.226945,0.263738,-0.002918,0.038642,-0.147208,-0.260159,41,42,-1,17,13,8,9,5,72,75,76,100,70,79,69,5,5,believ life death,true life death
404348,What is one coin?,What's this coin?,0,3.656829,1.382975,0.069016,5.065351,1.0,91.936365,0.371527,0.170819,0.182876,-0.193922,-0.147340,-0.279527,-0.397618,17,17,0,11,11,4,3,1,75,75,76,100,75,75,75,1,1,one coin,what 's coin
404349,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0,3.575980,1.313263,0.708233,16.438430,1.0,194.867266,1.190154,0.540202,0.737343,-0.018571,-0.155503,-0.382487,-0.162977,94,127,-33,26,27,17,25,1,30,35,34,100,36,37,35,0,0,approx annual cost live studi uic chicago indi...,littl hairfal problem want use hair style prod...


# Unigram features

#UNIGRAM

In [252]:
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import deepcopy
tfv_q1_unigram = TfidfVectorizer(min_df=3, 
max_features=10,
ngram_range=(1, 1), 
use_idf=1, 
smooth_idf=1, 
sublinear_tf=1,
)

tfv_q2_unigram = deepcopy(tfv_q1_unigram)

In [253]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data,data.is_duplicate,test_size=0.2,random_state=45)
y_train = y_train.values.astype('float32').reshape(-1, 1)
y_test= y_test.values.astype('float32').reshape(-1, 1)

In [254]:
q1_tfidf_train = tfv_q1_unigram.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_unigram.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_unigram.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_unigram.fit_transform(X_test.processed_q2.fillna(""))

In [255]:
q1_tfidf_train.shape

(323480, 10)

In [256]:
q2_tfidf_train.shape

(323480, 10)

In [257]:
from scipy import sparse
# obtain features by stacking the sparse matrices together
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

In [258]:
fs3_1_train.shape

(323480, 20)

In [259]:
fs3_1_test.shape

(80871, 20)

##Logistic on unigram

In [260]:
#train model
clf_unigram = sklearn.linear_model.LogisticRegression(tol=0.001, C=1.2, max_iter=100, solver='liblinear')
clf_unigram.fit(fs3_1_train, y_train)
 
#evaluation
predictions = clf_unigram.predict(fs3_1_test)
score = clf_unigram.score(fs3_1_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = clf_unigram.predict(fs3_1_test)
score = clf_unigram.score(fs3_1_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn =  63.73067886731791 %
Accuracy on test data with sklearn =  63.72123505335659 %
F1 score  0.5379045680443973


### SVM on unigram

In [261]:
q1_tfidf_train_svm = tfv_q1_unigram.fit_transform(X_train.processed_q1[:50000].fillna(""))
q2_tfidf_train_svm = tfv_q2_unigram.fit_transform(X_train.processed_q2[:50000].fillna(""))
q1_tfidf_test_svm = tfv_q1_unigram.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test_svm = tfv_q2_unigram.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train_svm = sparse.hstack((q1_tfidf_train_svm, q2_tfidf_train_svm))
fs3_1_test_svm = sparse.hstack((q1_tfidf_test_svm, q2_tfidf_test_svm))

In [262]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear')
clsvm.fit(fs3_1_train_svm, y_train[0:50000])
prediction_SVM_all = clsvm.predict(fs3_1_test_svm)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [264]:
score = clsvm.score(fs3_1_test_svm, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

Accuracy on test data with sklearn svm=  61.58820838124915 %
F1 score  0.49404497234349376


#Bigram

In [265]:

tfv_q1_bi = TfidfVectorizer(min_df=3, 
max_features=10, 
ngram_range=(2, 2), 
use_idf=1, 
smooth_idf=1, 
sublinear_tf=1,
)

tfv_q2_bi = deepcopy(tfv_q1_bi)

In [266]:
q1_tfidf_train = tfv_q1_bi.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_bi.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_bi.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_bi.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

In [267]:
fs3_1_test.shape

(80871, 20)

### Logistic regression on bigram features

In [268]:
#train model
clf_bigram = sklearn.linear_model.LogisticRegression(tol=0.001, C=1.2, max_iter=100, solver='liblinear')
clf_bigram.fit(fs3_1_train, y_train)
 
#evaluation
predictions = clf_bigram.predict(fs3_1_test)
score = clf_bigram.score(fs3_1_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = clf_bigram.predict(fs3_1_test)
score = clf_bigram.score(fs3_1_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn =  65.37189316186472 %
Accuracy on test data with sklearn =  65.57480431798791 %
F1 score  0.5642617557938848


### SVM on bigram features

In [269]:
q1_tfidf_train_svm = tfv_q1_bi.fit_transform(X_train.processed_q1[:50000].fillna(""))
q2_tfidf_train_svm = tfv_q2_bi.fit_transform(X_train.processed_q2[:50000].fillna(""))
q1_tfidf_test_svm = tfv_q1_bi.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test_svm = tfv_q2_bi.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train_svm = sparse.hstack((q1_tfidf_train_svm, q2_tfidf_train_svm))
fs3_1_test_svm = sparse.hstack((q1_tfidf_test_svm, q2_tfidf_test_svm))

In [270]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear')
clsvm.fit(fs3_1_train_svm, y_train[0:50000])
prediction_SVM_all = clsvm.predict(fs3_1_test_svm)
score = clsvm.score(fs3_1_train_svm,  y_train[0:50000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(fs3_1_test_svm, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn svm=  65.396 %
Accuracy on test data with sklearn svm=  65.49690247431094 %
F1 score  0.5602575086035515


#Trigram

In [271]:
tfv_q1_tri = TfidfVectorizer(min_df=3, 
max_features=10, 
ngram_range=(3, 3), 
use_idf=1, 
smooth_idf=1, 
sublinear_tf=1,
)

tfv_q2_tri = deepcopy(tfv_q1_tri)

In [272]:
q1_tfidf_train = tfv_q1_tri.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_tri.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_tri.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_tri.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

### Logistic on bigram features

In [273]:
#train model
clf_tri = sklearn.linear_model.LogisticRegression(tol=0.001, C=1.2, max_iter=100, solver='liblinear')
clf_tri.fit(fs3_1_train, y_train)
 
#evaluation
predictions = clf_tri.predict(fs3_1_test)
score = clf_tri.score(fs3_1_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = clf_tri.predict(fs3_1_test)
score = clf_tri.score(fs3_1_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn =  63.95913194015086 %
Accuracy on test data with sklearn =  63.99698284922902 %
F1 score  0.5076880047541213


### SVM on trigram features

In [ ]:
q1_tfidf_train_svm = tfv_q1_tri.fit_transform(X_train.processed_q1[:50000].fillna(""))
q2_tfidf_train_svm = tfv_q2_tri.fit_transform(X_train.processed_q2[:50000].fillna(""))
q1_tfidf_test_svm = tfv_q1_tri.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test_svm = tfv_q2_tri.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train_svm = sparse.hstack((q1_tfidf_train_svm, q2_tfidf_train_svm))
fs3_1_test_svm = sparse.hstack((q1_tfidf_test_svm, q2_tfidf_test_svm))

In [274]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear')
clsvm.fit(fs3_1_train_svm, y_train[0:50000])
prediction_SVM_all = clsvm.predict(fs3_1_test_svm)
score = clsvm.score(fs3_1_train_svm,  y_train[0:50000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(fs3_1_test_svm, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn svm=  65.396 %
Accuracy on test data with sklearn svm=  65.49690247431094 %
F1 score  0.5602575086035515


#UNIGRAM + BIGRAM + TRIGRAM

In [275]:
tfv_q1_all = TfidfVectorizer(min_df=3, 
max_features=20, 
ngram_range=(1, 3), 
use_idf=1, 
smooth_idf=1, 
sublinear_tf=1,
)

tfv_q2_all = deepcopy(tfv_q1_all)

In [276]:
q1_tfidf_train = tfv_q1_all.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_all.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_all.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_all.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

### Logistic on unigram + bigram + trigram features

In [277]:
#train model
clf_all = sklearn.linear_model.LogisticRegression(tol=0.001, C=1.2, max_iter=100, solver='liblinear')
clf_all.fit(fs3_1_train, y_train)
 
#evaluation
predictions = clf_all.predict(fs3_1_test)
score = clf_all.score(fs3_1_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = clf_all.predict(fs3_1_test)
score = clf_all.score(fs3_1_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn =  65.24236428836404 %
Accuracy on test data with sklearn =  65.36830260538389 %
F1 score  0.5914047570798305


### SVM on unigram + bigram + trigram features

In [278]:
q1_tfidf_train_svm = tfv_q1_tri.fit_transform(X_train.processed_q1[:50000].fillna(""))
q2_tfidf_train_svm = tfv_q2_tri.fit_transform(X_train.processed_q2[:50000].fillna(""))
q1_tfidf_test_svm = tfv_q1_tri.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test_svm = tfv_q2_tri.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train_svm = sparse.hstack((q1_tfidf_train_svm, q2_tfidf_train_svm))
fs3_1_test_svm = sparse.hstack((q1_tfidf_test_svm, q2_tfidf_test_svm))

In [279]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear')
clsvm.fit(fs3_1_train_svm, y_train[0:50000])
prediction_SVM_all = clsvm.predict(fs3_1_test_svm)
score = clsvm.score(fs3_1_train_svm,  y_train[0:50000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(fs3_1_test_svm, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on train data with sklearn svm=  64.068 %
Accuracy on test data with sklearn svm=  63.809029194643315 %
F1 score  0.506610953975886


#Basic Features

In [358]:
data_basic=data[fs_1]
label = data['is_duplicate']

In [359]:
nanIndices = np.argwhere(np.isnan(np.asarray(data_basic)))
print(nanIndices)

[]


In [370]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
X = scaler.fit_transform(np.asarray(data_basic))

In [371]:
y = np.asarray(label)
print(X.shape)
print(y.shape)

(404351, 8)
(404351,)


In [372]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=45)

X_train.shape

(323480, 8)

In [373]:
classifier = sklearn.linear_model.LogisticRegression(tol=0.0001, C=1.2, max_iter=1000, solver='liblinear')
classifier.fit(X_train, y_train)
score = classifier.score(X_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = classifier.predict(X_test)
score = classifier.score(X_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

Accuracy on train data with sklearn =  65.68226783726969 %
Accuracy on test data with sklearn =  65.90990589951899 %
F1 score  0.6465143820879312


#Fuzzy Features 

In [364]:
data_fuzzy=data[fs_2]
label = data['is_duplicate']

In [365]:
nanIndices = np.argwhere(np.isnan(np.asarray(data_fuzzy)))
print(nanIndices)

[]


In [366]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
X = scaler.fit_transform(np.asarray(data_fuzzy))

In [367]:
y = np.asarray(label)
print(X.shape)
print(y.shape)

(404351, 7)
(404351,)


In [368]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=45)

X_train.shape

(323480, 7)

## Logistic regression

In [369]:
classifier = sklearn.linear_model.LogisticRegression(tol=0.0001, C=1.2, max_iter=1000, solver='liblinear')
classifier.fit(X_train, y_train)
score = classifier.score(X_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = classifier.predict(X_test)
score = classifier.score(X_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

Accuracy on train data with sklearn =  66.07178187214046 %
Accuracy on test data with sklearn =  66.39586501959911 %
F1 score  0.6583675209135702


## Word to vec features

In [298]:
data_w2v=data[fs4_1]
label = data['is_duplicate']

In [303]:
data_w2v["cosine_distance"] = data_w2v["cosine_distance"].fillna(data_w2v["cosine_distance"].median(skipna=True))
data_w2v["braycurtis_distance"] = data_w2v["braycurtis_distance"].fillna(data_w2v["braycurtis_distance"].median(skipna=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [304]:
nanIndices = np.argwhere(np.isnan(np.asarray(data_fuzzy)))
print(nanIndices)

[]


In [305]:
X = scaler.fit_transform(np.asarray(data_w2v))
y = np.asarray(label)
print(X.shape)
print(y.shape)

(404351, 7)
(404351,)


In [306]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=45)

In [307]:
classifier = sklearn.linear_model.LogisticRegression(tol=0.0001, C=1.2, max_iter=1000, solver='liblinear')
classifier.fit(X_train, y_train)
score = classifier.score(X_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = classifier.predict(X_test)
score = classifier.score(X_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

Accuracy on train data with sklearn =  67.05731420798813 %
Accuracy on test data with sklearn =  67.2725698952653 %
F1 score  0.6717785510833124


## Word to vec and bigram features


In [379]:
data_f=data[fs4_1+["processed_q1"]+["processed_q2"]]
label = data['is_duplicate']
data_f["cosine_distance"] = data_f["cosine_distance"].fillna(data_f["cosine_distance"].median(skipna=True))
data_f["braycurtis_distance"] = data_f["braycurtis_distance"].fillna(data_f["braycurtis_distance"].median(skipna=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [380]:
y = np.asarray(label)

In [381]:
X_train,X_test,y_train,y_test=train_test_split(data_f,y,test_size=0.2,random_state=45)

In [382]:
q1_tfidf_train = tfv_q1_bi.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_bi.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_bi.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_bi.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

In [383]:
X_train=X_train[fs4_1]

In [384]:
X_sc = scaler.fit_transform(np.asarray(X_train))

In [385]:
X_test=X_test[fs4_1]

In [386]:
X_sc_test = scaler.fit_transform(np.asarray(X_test))

In [387]:
features_train=sparse.hstack((fs3_1_train, X_sc))
features_test=sparse.hstack((fs3_1_test, X_sc_test))

### logistic regression

In [388]:
classifier = sklearn.linear_model.LogisticRegression(tol=0.0001, C=1.2, max_iter=1000, solver='liblinear')
classifier.fit(features_train, y_train)
score = classifier.score(features_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = classifier.predict(features_test)
score = classifier.score(features_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

Accuracy on train data with sklearn =  68.6373191541981 %
Accuracy on test data with sklearn =  68.8058760247802 %
F1 score  0.6859033467667521


### Fuzzy features And word to vec features and Bigram features

In [391]:
data_fwb=data[fs_2+ fs4_1+["processed_q1"]+["processed_q2"]]
label = data['is_duplicate']
data_fwb["cosine_distance"] = data_fwb["cosine_distance"].fillna(data_fwb["cosine_distance"].median(skipna=True))
data_fwb["braycurtis_distance"] = data_fwb["braycurtis_distance"].fillna(data_fwb["braycurtis_distance"].median(skipna=True))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [392]:
y = np.asarray(label)
X_train,X_test,y_train,y_test=train_test_split(data_fwb,y,test_size=0.2,random_state=45)

In [393]:
q1_tfidf_train = tfv_q1_bi.fit_transform(X_train.processed_q1.fillna(""))
q2_tfidf_train = tfv_q2_bi.fit_transform(X_train.processed_q2.fillna(""))
q1_tfidf_test = tfv_q1_bi.fit_transform(X_test.processed_q1.fillna(""))
q2_tfidf_test = tfv_q2_bi.fit_transform(X_test.processed_q2.fillna(""))
fs3_1_train = sparse.hstack((q1_tfidf_train, q2_tfidf_train))
fs3_1_test = sparse.hstack((q1_tfidf_test, q2_tfidf_test))

In [394]:
X_train=X_train[fs4_1 + fs_2]
X_sc = scaler.fit_transform(np.asarray(X_train))
X_test=X_test[fs4_1 + fs_2]
X_sc_test = scaler.fit_transform(np.asarray(X_test))

In [395]:
features_train=sparse.hstack((fs3_1_train, X_sc))
features_test=sparse.hstack((fs3_1_test, X_sc_test))

In [396]:
classifier = sklearn.linear_model.LogisticRegression(tol=0.0001, C=1.2, max_iter=1000, solver='liblinear')
classifier.fit(features_train, y_train)
score = classifier.score(features_train, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")

predictions = classifier.predict(features_test)
score = classifier.score(features_test, y_test)
print("Accuracy on test data with sklearn = ",score*100 ,"%")
print("F1 score ",f1_score(y_test, predictions, average='weighted'))

Accuracy on train data with sklearn =  68.68647211574131 %
Accuracy on test data with sklearn =  68.89985285207305 %
F1 score  0.687906860725369


## Random forest

In [351]:
import time
from sklearn.ensemble import RandomForestClassifier

clfr = RandomForestClassifier(n_estimators = 5)
start_time = time.time()
clfr.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfr.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfr.predict(X_sc_test)
score = clfr.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  6.827272176742554
Accuracy on train data with sklearn =  96.37844689007048 %
Accuracy on test data with sklearn rf=  68.35824955793795 %
F1 score  0.6825452923312076


In [352]:
clfr = RandomForestClassifier(n_estimators = 10)
start_time = time.time()
clfr.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfr.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfr.predict(X_sc_test)
score = clfr.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  13.465991497039795
Accuracy on train data with sklearn =  98.25120563867937 %
Accuracy on test data with sklearn rf=  69.45134844381793 %
F1 score  0.6866933845343334


In [353]:
clfr = RandomForestClassifier(n_estimators = 20)
start_time = time.time()
clfr.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfr.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfr.predict(X_sc_test)
score = clfr.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  27.27974033355713
Accuracy on train data with sklearn =  99.33689872635092 %
Accuracy on test data with sklearn rf=  70.1450458137033 %
F1 score  0.6971765921377587


## Decision Tree

In [354]:
from sklearn.tree import DecisionTreeClassifier

clfd = DecisionTreeClassifier(criterion ="entropy", max_depth = 10)
start_time = time.time()
clfd.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfd.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfd.predict(X_sc_test)
score = clfd.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  6.200486183166504
Accuracy on train data with sklearn =  71.2489180165698 %
Accuracy on test data with sklearn rf=  70.92530078767419 %
F1 score  0.712448100204165


In [400]:
from sklearn.tree import DecisionTreeClassifier

clfd = DecisionTreeClassifier(criterion ="gini", max_depth = 15)
start_time = time.time()
clfd.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfd.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfd.predict(X_sc_test)
score = clfd.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  5.952317714691162
Accuracy on train data with sklearn =  75.4572152837888 %
Accuracy on test data with sklearn rf=  70.27735529423403 %
F1 score  0.7062581238221107


In [355]:
clfd = DecisionTreeClassifier(criterion ="entropy", max_depth = 15)
start_time = time.time()
clfd.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfd.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfd.predict(X_sc_test)
score = clfd.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  8.494362115859985
Accuracy on train data with sklearn =  74.30783974279707 %
Accuracy on test data with sklearn rf=  70.5432107924967 %
F1 score  0.708033183816975


In [397]:
clfd = DecisionTreeClassifier(criterion ="entropy", max_depth = 20)
start_time = time.time()
clfd.fit(X_sc, y_train)
end_time = time.time()
print("Training time: ", end_time-start_time)
score = clfd.score(X_sc, y_train)
print("Accuracy on train data with sklearn = ",score*100 ,"%")
y_train_pred_rf = clfd.predict(X_sc_test)
score = clfd.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn rf= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, y_train_pred_rf, average='weighted'))

Training time:  10.292390584945679
Accuracy on train data with sklearn =  79.34740942252999 %
Accuracy on test data with sklearn rf=  69.24731980561634 %
F1 score  0.6955448562815879


##SVM

In [357]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear')
clsvm.fit(X_sc[0:100000], y_train[0:100000])
prediction_SVM_all = clsvm.predict(X_sc_test)
score = clsvm.score(X_sc[0:100000],  y_train[0:100000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

Accuracy on train data with sklearn svm=  67.56299999999999 %
Accuracy on test data with sklearn svm=  67.60025225358905 %
F1 score  0.6777150387816205


In [402]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear',C=0.01)
clsvm.fit(X_sc[0:100000], y_train[0:100000])
prediction_SVM_all = clsvm.predict(X_sc_test)
score = clsvm.score(X_sc[0:100000],  y_train[0:100000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

Accuracy on train data with sklearn svm=  67.542 %
Accuracy on test data with sklearn svm=  67.63363875802203 %
F1 score  0.6781472359969822


In [403]:
from sklearn import svm
clsvm = svm.SVC(kernel='linear',C=0.1)
clsvm.fit(X_sc[0:100000], y_train[0:100000])
prediction_SVM_all = clsvm.predict(X_sc_test)
score = clsvm.score(X_sc[0:100000],  y_train[0:100000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

Accuracy on train data with sklearn svm=  67.572 %
Accuracy on test data with sklearn svm=  67.59530610478416 %
F1 score  0.6777502213413245


In [401]:
from sklearn import svm
clsvm = svm.SVC(kernel='rbf')
clsvm.fit(X_sc[0:50000], y_train[0:50000])
prediction_SVM_all = clsvm.predict(X_sc_test)
score = clsvm.score(X_sc[0:50000],  y_train[0:50000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))

Accuracy on train data with sklearn svm=  70.822 %
Accuracy on test data with sklearn svm=  70.7769163235276 %
F1 score  0.70815675599594


In [ ]:
from sklearn import svm
clsvm = svm.SVC(kernel='rbf')
clsvm.fit(X_sc[0:50000], y_train[0:50000])
prediction_SVM_all = clsvm.predict(X_sc_test)
score = clsvm.score(X_sc[0:50000],  y_train[0:50000])
print("Accuracy on train data with sklearn svm= ",score*100 ,"%")
score = clsvm.score(X_sc_test, y_test)
print("Accuracy on test data with sklearn svm= ",score*100 ,"%")
print("F1 score ",f1_score(y_test, prediction_SVM_all, average='weighted'))